In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import collections
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1040,tt1596365,17000000,127730736,The Woman in Black,Daniel Radcliffe|CiarÃ¡n Hinds|Sidney Johnston...,James Watkins,What did they see?,"The story follows a young lawyer, Arthur Kipps...",95,Drama|Horror|Thriller,Hammer Film Productions|UK Film Council|Allian...,2/1/2012,6.1,2012
422,tt1320261,112000000,237382724,Gulliver's Travels,Emily Blunt|Jason Segel|Jack Black|Amanda Peet...,Rob Letterman,Something big is going down.,Travel writer Lemuel Gulliver takes an assignm...,85,Comedy,Twentieth Century Fox Film Corporation|Dune En...,12/25/2010,5.0,2010
889,tt0286106,72000000,408247917,Signs,Mel Gibson|Joaquin Phoenix|Rory Culkin|Abigail...,M. Night Shyamalan,Itâ€™s not like they didnâ€™t warn us.,A family living on a farm finds mysterious cro...,106,Drama|Thriller|Science Fiction|Mystery,"Kennedy/Marshall Company, The|Touchstone Pictu...",8/2/2002,6.3,2002
1822,tt0134847,23000000,53187659,Pitch Black,Vin Diesel|Radha Mitchell|Cole Hauser|Rhiana G...,David Twohy,Don't be afraid of the dark. Be afraid of what...,After crash-landing on a seemingly lifeless pl...,109,Thriller|Science Fiction|Action,Interscope Communications|Polygram Filmed Ente...,2/18/2000,6.6,2000
1288,tt1821549,12000000,17654912,Nebraska,Bruce Dern|Will Forte|Bob Odenkirk|June Squibb...,Alexander Payne,Life's not about winning or losing. It's about...,"An aging, booze-addled father takes a trip fro...",115,Drama|Adventure,Paramount Vantage|Bona Fide Productions,9/21/2013,7.3,2013


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.describe(include = ['object'])

,imdb_id,original_title,cast,director,tagline,overview,genres,production_companies,release_date
count,1889,1889,1889,1889,1889,1889,1889,1889,1889
unique,1889,1887,1878,957,1880,1889,652,1624,1435
top,tt0417741,The Gift,Elijah Wood|Ian McKellen|Viggo Mortensen|Liv T...,Steven Soderbergh,What could go wrong?,Following the death of his employer and mentor...,Comedy,New Line Cinema,12/25/2014
freq,1,2,3,13,2,1,141,21,7


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [6]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# код получения столбца profit
data['profit'] = data['revenue'] - data['budget']

# разделяем жанры
data_genres = data
data_genres['new_genres'] = data_genres.genres.str.split('|')
data_genres = data_genres.explode('new_genres')

# разделяем режиссёров
data_director = data
data_director['new_director'] = data_director.director.str.split('|')
data_director = data_director.explode('new_director')

# DF с разделёнными жанрами и режиссёрами
dir_gen = data_genres
dir_gen['new_director'] = dir_gen.director.str.split('|')
dir_gen = dir_gen.explode('new_director')

# разделяем актёрский состав
data_actors = data
data_actors['actors'] = data_actors.cast.str.split('|')
data_actors = data_actors.explode('actors')

# DF с разделёнными жанрами и актёрами
act_gen = data_genres
act_gen['actors'] = act_gen.cast.str.split('|')
act_gen = act_gen.explode('actors')

# разделяем киностудии
data_companies = data
data_companies['new_companies'] = data_companies.production_companies.str.split('|')
data_companies = data_companies.explode('new_companies')

# создаём колонку release_month
rel_month = data
rel_month['release_date'] = rel_month.release_date.str.split('/')
rel_month['release_month'] = rel_month['release_date'].apply(lambda x: x[0])

# DF c разделёнными режиссёрами и с колонкой release_month
rel_month_dir = rel_month
rel_month_dir['new_director'] = rel_month_dir.director.str.split('|')
rel_month_dir = rel_month_dir.explode('new_director')

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [7]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [8]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_genres,new_director,actors,new_companies,release_month
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,"[5, 11, 2011]",6.3,2011,641683000,"[Adventure, Action, Fantasy]",[Rob Marshall],"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...","[Walt Disney Pictures, Jerry Bruckheimer Films...",5


ВАРИАНТ 2

In [9]:
# можно добавлять разные варианты решения
data.groupby(['original_title'])[['budget']].max().sort_values('budget', ascending=False)

,budget
original_title,
Pirates of the Caribbean: On Stranger Tides,380000000
Pirates of the Caribbean: At World's End,300000000
Avengers: Age of Ultron,280000000
Tangled,260000000
John Carter,260000000
...,...
Paranormal Activity 3,5000000
Oculus,5000000
Paranormal Activity 4,5000000


# 2. Какой из фильмов самый длительный (в минутах)?

In [10]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '1157. Gods and Generals (tt0279111)'

In [11]:
data[data.runtime == data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_genres,new_director,actors,new_companies,release_month
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,"[2, 21, 2003]",5.8,2003,-43076064,"[Drama, History, War]",[Ronald F. Maxwell],"[Stephen Lang, Jeff Daniels, Robert Duvall, Ke...","[Turner Pictures, Antietam Filmworks]",2


# 3. Какой из фильмов самый короткий (в минутах)?





In [12]:
answers['3'] = '768. Winnie the Pooh (tt1449283)'

In [13]:
data[data.runtime == data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_genres,new_director,actors,new_companies,release_month
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,"[4, 13, 2011]",6.8,2011,-15540000,"[Animation, Family]","[Stephen Anderson, Don Hall]","[Jim Cummings, Travis Oates, Jim Cummings, Bud...","[Walt Disney Pictures, Walt Disney Animation S...",4


# 4. Какова средняя длительность фильмов?


In [14]:
answers['4'] = '110'

In [15]:
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [16]:
answers['5'] = '107'

In [17]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [18]:
answers['6'] = '239. Avatar (tt0499549)'

In [19]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[data.profit == data.profit.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_genres,new_director,actors,new_companies,release_month
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,"[12, 10, 2009]",7.1,2009,2544505847,"[Action, Adventure, Fantasy, Science Fiction]",[James Cameron],"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Ingenious Film Partners, Twentieth Century Fo...",12


# 7. Какой фильм самый убыточный? 

In [20]:
answers['7'] = '1245. The Lone Ranger (tt1210819)'

In [21]:
data[data.profit == data.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_genres,new_director,actors,new_companies,release_month
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,"[7, 3, 2013]",6.0,2013,-165710090,"[Action, Adventure, Western]",[Gore Verbinski],"[Johnny Depp, Armie Hammer, William Fichtner, ...","[Walt Disney Pictures, Jerry Bruckheimer Films...",7


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [22]:
answers['8'] = '1478'

In [23]:
data[data.profit > 0].count()

imdb_id                 1478
budget                  1478
revenue                 1478
original_title          1478
cast                    1478
director                1478
tagline                 1478
overview                1478
runtime                 1478
genres                  1478
production_companies    1478
release_date            1478
vote_average            1478
release_year            1478
profit                  1478
new_genres              1478
new_director            1478
actors                  1478
new_companies           1478
release_month           1478
dtype: int64

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [24]:
answers['9'] = '599. The Dark Knight (tt0468569)'

In [25]:
data[data.release_year == 2008].sort_values(['revenue'],ascending=False).head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_genres,new_director,actors,new_companies,release_month
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,"[7, 16, 2008]",8.1,2008,816921825,"[Drama, Action, Crime, Thriller]",[Christopher Nolan],"[Christian Bale, Michael Caine, Heath Ledger, ...","[DC Comics, Legendary Pictures, Warner Bros., ...",7
603,tt0367882,185000000,786636033,Indiana Jones and the Kingdom of the Crystal S...,Harrison Ford|Cate Blanchett|Shia LaBeouf|Ray ...,Steven Spielberg,The adventure continues . . .,"Set during the Cold War, the Soviets â€“ led b...",122,Adventure|Action,Lucasfilm|Paramount Pictures,"[5, 21, 2008]",5.6,2008,601636033,"[Adventure, Action]",[Steven Spielberg],"[Harrison Ford, Cate Blanchett, Shia LaBeouf, ...","[Lucasfilm, Paramount Pictures]",5
606,tt0441773,130000000,631744560,Kung Fu Panda,Jack Black|Dustin Hoffman|Angelina Jolie|Jacki...,Mark Osborne|John Stevenson,Prepare for awesomeness.,"When the Valley of Peace is threatened, lazy P...",90,Animation|Family,DreamWorks Animation,"[6, 4, 2008]",6.8,2008,501744560,"[Animation, Family]","[Mark Osborne, John Stevenson]","[Jack Black, Dustin Hoffman, Angelina Jolie, J...",[DreamWorks Animation],6
621,tt0448157,150000000,624029371,Hancock,Will Smith|Charlize Theron|Jason Bateman|Jae H...,Peter Berg,Bad Behavior. Bad Attitude. Real Hero.,Hancock is a down-and-out superhero who's forc...,92,Fantasy|Action,Columbia Pictures|Weed Road Pictures|Forward P...,"[7, 1, 2008]",6.0,2008,474029371,"[Fantasy, Action]",[Peter Berg],"[Will Smith, Charlize Theron, Jason Bateman, J...","[Columbia Pictures, Weed Road Pictures, Forwar...",7
607,tt0795421,52000000,609841637,Mamma Mia!,Meryl Streep|Pierce Brosnan|Amanda Seyfried|Co...,Phyllida Lloyd,Take a trip down the aisle you'll never forget,"Set on an idyllic Greek island, the plot serve...",108,Comedy|Romance,Universal Pictures|Internationale Filmprodukti...,"[6, 30, 2008]",6.2,2008,557841637,"[Comedy, Romance]",[Phyllida Lloyd],"[Meryl Streep, Pierce Brosnan, Amanda Seyfried...","[Universal Pictures, Internationale Filmproduk...",6


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [26]:
answers['10'] = '1245. The Lone Ranger (tt1210819)'

In [27]:
data[(data.release_year >= 2012) & (data.release_year <= 2014)].sort_values(['profit']).head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_genres,new_director,actors,new_companies,release_month
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,"[7, 3, 2013]",6.0,2013,-165710090,"[Action, Adventure, Western]",[Gore Verbinski],"[Johnny Depp, Armie Hammer, William Fichtner, ...","[Walt Disney Pictures, Jerry Bruckheimer Films...",7
1214,tt0790736,130000000,61648500,R.I.P.D.,Jeff Bridges|Ryan Reynolds|Kevin Bacon|Stephan...,Robert Schwentke,To protect and serve the living,A recently slain cop joins a team of undead po...,96,Fantasy|Action|Comedy|Crime,Universal Pictures|Original Film|Dark Horse En...,"[7, 18, 2013]",5.3,2013,-68351500,"[Fantasy, Action, Comedy, Crime]",[Robert Schwentke],"[Jeff Bridges, Ryan Reynolds, Kevin Bacon, Ste...","[Universal Pictures, Original Film, Dark Horse...",7
1007,tt1374992,60000000,8106475,Upside Down,Kirsten Dunst|Jim Sturgess|Timothy Spall|James...,Juan Diego Solanas,Two worlds. One future.,In an alternate universe where twinned worlds ...,104,Romance|Science Fiction|Drama|Fantasy,Onyx Films|Studio 37|Jouror Productions|France...,"[9, 27, 2012]",6.1,2012,-51893525,"[Romance, Science Fiction, Drama, Fantasy]",[Juan Diego Solanas],"[Kirsten Dunst, Jim Sturgess, Timothy Spall, J...","[Onyx Films, Studio 37, Jouror Productions, Fr...",9
1302,tt0884726,70000000,18662027,Legends of Oz: Dorothy's Return,Lea Michele|Dan Aykroyd|Patrick Stewart|Hugh D...,Dan St. Pierre,There's trouble in OZ...,"Dorothy wakes up in post-tornado Kansas, only ...",88,Animation|Music|Family,Prana Animation Studios|Summertime Entertainment,"[6, 13, 2013]",6.1,2013,-51337973,"[Animation, Music, Family]",[Dan St. Pierre],"[Lea Michele, Dan Aykroyd, Patrick Stewart, Hu...","[Prana Animation Studios, Summertime Entertain...",6
1235,tt1308729,55000000,9489829,Bullet to the Head,Sylvester Stallone|Sung Kang|Sarah Shahi|Adewa...,Walter Hill,Revenge Never Gets Old.,"After watching their respective partners die, ...",92,Action|Crime|Thriller,Dark Castle Entertainment|After Dark Films|IM ...,"[1, 31, 2013]",5.2,2013,-45510171,"[Action, Crime, Thriller]",[Walter Hill],"[Sylvester Stallone, Sung Kang, Sarah Shahi, A...","[Dark Castle Entertainment, After Dark Films, ...",1


# 11. Какого жанра фильмов больше всего?

In [28]:
answers['11'] = 'Drama'

In [29]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
# используем DF (data_genres) с разделёнными жанрами 
data_genres.new_genres.describe()

count      5107
unique       19
top       Drama
freq        782
Name: new_genres, dtype: object

ВАРИАНТ 2

In [30]:
data_genres.new_genres.value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: new_genres, dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [31]:
answers['12'] = 'Drama'

In [32]:
# используем DF (data_genres) с разделёнными жанрами 
data_genres[data_genres.profit > 0].new_genres.value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: new_genres, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [33]:
answers['13'] = 'Peter Jackson'

In [34]:
# используем DF (data_director) с разделёнными режиссёрами 
data_director.groupby(['new_director'])[['revenue']].sum().sort_values('revenue', ascending=False)

,revenue
new_director,
Peter Jackson,6490593685
Christopher Nolan,4167548502
David Yates,4154295625
Michael Bay,3886938960
J.J. Abrams,3579169916
...,...
David MichÃ´d,2295423
Steven Shainberg,2281089
Paul Schrader,2062066


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [35]:
answers['14'] = 'Robert Rodriguez'

In [36]:
# используем DF (dir_gen) с разделёнными жанрами и режиссёрами
dir_gen[dir_gen.new_genres == 'Action'].new_director.value_counts()

Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
                     ..
John McTiernan        1
Shane Black           1
Jon Avnet             1
Michael Spierig       1
Sanjay Gadhvi         1
Name: new_director, Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [37]:
answers['15'] = 'Chris Hemsworth'

In [38]:
# используем DF (data_actors) с разделёнными актёрами
data_actors[data_actors.release_year == 2012].groupby(['actors'])[['revenue']].sum()\
.sort_values('revenue', ascending=False).head()

,revenue
actors,
Chris Hemsworth,2027450773
Denis Leary,1629460639
Anne Hathaway,1522851057
Robert Downey Jr.,1519557910
Mark Ruffalo,1519557910


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [39]:
answers['16'] = 'Matt Damon'

In [40]:
# используем DF (data_actors) с разделёнными актёрами
data_actors[data_actors.budget > data_actors.budget.mean()].actors.value_counts()

Matt Damon            18
Adam Sandler          17
Angelina Jolie        16
Eddie Murphy          15
Tom Cruise            15
                      ..
Nick Robinson          1
Denis O'Hare           1
StÃ©phanie Lagarde     1
Sean Connery           1
Daniela Ruah           1
Name: actors, Length: 1505, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [41]:
answers['17'] = 'Action'

In [42]:
# используем DF с разделёнными жанрами и актёрами
act_gen[act_gen.actors == 'Nicolas Cage'].new_genres.value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Mystery             3
Family              3
Animation           3
History             2
Horror              1
War                 1
Romance             1
Name: new_genres, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [43]:
answers['18'] = '925. K-19: The Widowmaker (tt0267626)'

In [44]:
# создаём DF по кинокомпании Paramount Pictures и используем ранее созданный data_companies
# с разделёнными киностудиями

# встречаются другие дочерние Paramount, в данном случае, как понимаю должна быть именно Paramount Pictures
# paramount = data_companies[data_companies.new_companies.str.contains('Paramount')]
paramount = data_companies[data_companies.new_companies == 'Paramount Pictures']
paramount[paramount.profit == paramount.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,new_genres,new_director,actors,new_companies
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,7/19/2002,6.0,2002,-64831034,"[Thriller, Drama, History]",[Kathryn Bigelow],"[Harrison Ford, Liam Neeson, Peter Sarsgaard, ...",Paramount Pictures


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [45]:
answers['19'] = '2015'

In [46]:
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [47]:
answers['20'] = '2014'

In [48]:
# создаём DF по кинокомпании Warner Bros по 'Warner', т.к. не совсем понятно, это разное
# написание названия или дочерние компании и используем ранее созданный data_companies
# с разделёнными киностудиями

warner_bros = data_companies[data_companies.new_companies.str.contains('Warner')]
#удаляем задвоенные фильмы, над которыми трудились по нескольку подразделений Warner...
#warner_bros.drop_duplicates(subset = 'imdb_id', keep = 'first', inplace = True) - не стала делать,
#в данном случае правильный ответ не меняется
warner_bros.groupby('release_year')[['profit']].sum().sort_values('profit', ascending = False)

,profit
release_year,
2014,2292949646
2007,2208217007
2008,2134595031
2010,1974712985
2011,1871393682
2003,1844008221
2009,1822454136
2013,1636453400
2004,1631933725


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [49]:
answers['21'] = 'сентябрь'

In [50]:
# используем DF rel_month с колонкой релизов по месяцам
rel_month.release_month.value_counts()

9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: release_month, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [51]:
answers['22'] = '450'

In [52]:
rel_month.query('release_month in ["6", "7","8"]').release_month.count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [53]:
answers['23'] = 'Peter Jackson'

In [54]:
# используем DF (rel_month_dir) c разделёнными режиссёрами и с колонкой release_month
rel_month_dir.query('release_month in ["12","1", "2"]').new_director.value_counts()

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
Martin Scorsese      4
Adam Shankman        4
                    ..
Rob Letterman        1
Neil Nightingale     1
Russell Crowe        1
Justin Lin           1
Brad Peyton          1
Name: new_director, Length: 358, dtype: int64

In [55]:
rel_month_dir[rel_month_dir.release_month.isin(["12", "1","2"])].new_director.value_counts()

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
Martin Scorsese      4
Adam Shankman        4
                    ..
Rob Letterman        1
Neil Nightingale     1
Russell Crowe        1
Justin Lin           1
Brad Peyton          1
Name: new_director, Length: 358, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [56]:
answers['24'] = 'Four By Two Productions'

In [57]:
# используем DF с разделёнными киностудиями, содаём новую колонку (title_length) с кол-ом
# символов из названий фильмов
data_companies['title_length'] = data_companies['original_title'].apply(len)
data_companies.groupby('new_companies')[['title_length']].max().sort_values('title_length', ascending=False)

,title_length
new_companies,
Twentieth Century Fox Film Corporation,83
Four By Two Productions,83
Walt Disney,62
Walden Media,62
21 Laps Entertainment,59
...,...
Everest Entertainment,3
Berlanti Productions,3
Global Entertainment Group,2


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [58]:
answers['25'] = 'Midnight Picture Show'

In [68]:
# создаём колонку с количеством слов из описания
data_companies['overview_len'] = data_companies['overview'].apply(lambda x: x.split()).apply(len)
data_companies = data_companies.groupby('new_companies').overview_len.mean()
data_companies[data_companies == data_companies.max()]

new_companies
Midnight Picture Show    175.0
Name: overview_len, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [60]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

In [61]:
data.loc[data['vote_average']>data.quantile(0.99, numeric_only=True)['vote_average']]['original_title']

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

In [62]:
# находим через квантиль нужный диапазон
quantile_rating = data.quantile(0.99, numeric_only=True)['vote_average']
data[data.vote_average >= quantile_rating].original_title

9                                              Inside Out
28                                              Spotlight
34                                                   Room
118                                          Interstellar
119                               Guardians of the Galaxy
124                                            Big Hero 6
125                                    The Imitation Game
128                                             Gone Girl
138                              The Grand Budapest Hotel
155                              The Theory of Everything
177                                The Fault in Our Stars
283                                            Mr. Nobody
316                                              3 Idiots
370                                             Inception
496     The Lord of the Rings: The Fellowship of the Ring
599                                       The Dark Knight
863                 The Lord of the Rings: The Two Towers
872           

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [63]:
answers['27'] = 'Daniel Radcliffe, Rupert Grint'

In [64]:

data['cast'] = data['cast'].str.split('|')
data['pairs'] = data['cast'].apply(lambda s: list(combinations(s, 2)))
data = data.explode('pairs')
collections.Counter(data['pairs']).most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

ВАРИАНТ 2

# Submission

In [65]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1245. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': '925. K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe, Rupert Grint'}

In [66]:
# и убедиться что ни чего не пропустил)
len(answers)

27